# Plot counts per lineage through time

## Imports

In [ ]:
import altair as alt
import pandas as pd

## Configuration

In [ ]:
min_date = "2019-10-01"

## Load metadata

In [ ]:
def load_lineage_dates(metadata_file, lineage):
    df = pd.read_csv(metadata_file, sep="\t").dropna(subset=["date"])
    df["lineage"] = lineage
    dates = df.loc[~(df["date"].str.contains("X")), ["lineage", "date", "region"]]
    
    return dates

In [ ]:
h3n2_dates = load_lineage_dates("../data/h3n2/metadata.tsv", "H3N2")

In [ ]:
h1n1pdm_dates = load_lineage_dates("../data/h1n1pdm/metadata.tsv", "H1N1pdm")

In [ ]:
vic_dates = load_lineage_dates("../data/vic/metadata.tsv", "Vic")

In [ ]:
yam_dates = load_lineage_dates("../data/yam/metadata.tsv", "Yam")

In [ ]:
#dates = pd.concat([h3n2_dates, h1n1pdm_dates, vic_dates])
dates = pd.concat([h3n2_dates, h1n1pdm_dates, vic_dates, yam_dates])

In [ ]:
dates = dates.query("date != '36-09-05'").copy()

In [ ]:
dates.head()

In [ ]:
dates["date"].max()

In [ ]:
dates.shape

In [ ]:
recent_dates = dates[dates["date"] > min_date].copy()

In [ ]:
recent_dates["date"] = pd.to_datetime(recent_dates["date"])

In [ ]:
recent_dates.shape

In [ ]:
regions = [region for region in sorted(recent_dates["region"].drop_duplicates().values) if region != "?"]

In [ ]:
binned_counts = recent_dates.set_index("date").groupby("lineage").resample("1MS").count().rename(columns={"lineage": "samples"}).reset_index()

In [ ]:
binned_counts

In [ ]:
alt.Chart(binned_counts).mark_line(point=alt.OverlayMarkDef(size=100)).encode(
    x=alt.X("yearmonth(date):T", title="Date"),
    y=alt.Y("samples:Q", title="Number of samples"),
    color=alt.Color("lineage:N", sort=["H3N2", "H1N1pdm", "Vic", "Yam"], title="Lineage"),
    tooltip=["lineage:N", "date:T", "samples:Q"],
).configure_axis(
    labelFontSize=14,
    titleFontSize=14,
).configure_legend(
    labelFontSize=14,
    titleFontSize=14,
).properties(
    width=900,
    height=400,
)

In [ ]:
binned_counts_h1n1pdm = recent_dates.set_index("date").query(
    "(lineage == 'H1N1pdm') & (date >= '2021-01-01')"
).groupby(
    "region"
).resample("1MS").count().rename(columns={"region": "samples"}).reset_index()

In [ ]:
binned_counts_h1n1pdm

In [ ]:
alt.Chart(binned_counts_h1n1pdm).mark_line(point=alt.OverlayMarkDef(size=100)).encode(
    x=alt.X("yearmonth(date):T", title="Date"),
    y=alt.Y("samples:Q", title="Number of samples"),
    color=alt.Color("region:N", title="Region", scale=alt.Scale(domain=regions, scheme='category10')),
    tooltip=["region:N", "date:T", "samples:Q"],
).configure_axis(
    labelFontSize=14,
    titleFontSize=14,
).configure_legend(
    labelFontSize=14,
    titleFontSize=14,
).properties(
    width=900,
    height=300,
)

In [ ]:
binned_counts_h3n2 = recent_dates.set_index("date").query(
    "(lineage == 'H3N2') & (date >= '2019-11-01')"
).groupby(
    "region"
).resample("1MS").count().rename(columns={"region": "samples"}).reset_index()

In [ ]:
binned_counts_h3n2.query("region == 'China'")

In [ ]:
alt.Chart(binned_counts_h3n2).mark_line(point=alt.OverlayMarkDef(size=100)).encode(
    x=alt.X("yearmonth(date):T", title="Date"),
    y=alt.Y("samples:Q", title="Number of samples"),
    color=alt.Color("region:N", title="Region", scale=alt.Scale(domain=regions, scheme='category10')),
    tooltip=["region:N", "date:T", "samples:Q"],
).configure_axis(
    labelFontSize=14,
    titleFontSize=14,
).configure_legend(
    labelFontSize=14,
    titleFontSize=14,
).properties(
    width=900,
    height=300,
)

In [ ]:
binned_counts_vic = recent_dates.set_index("date").query(
    "(lineage == 'Vic') & (date >= '2021-01-01')"
).groupby(
    "region"
).resample("1MS").count().rename(columns={"region": "samples"}).reset_index()

In [ ]:
alt.Chart(binned_counts_vic).mark_line(point=alt.OverlayMarkDef(size=100)).encode(
    x=alt.X("yearmonth(date):T", title="Date"),
    y=alt.Y("samples:Q", title="Number of samples"),
    color=alt.Color("region:N", title="Region", scale=alt.Scale(domain=regions, scheme='category10')),
    tooltip=["region:N", "date:T", "samples:Q"],
).configure_axis(
    labelFontSize=14,
    titleFontSize=14,
).configure_legend(
    labelFontSize=14,
    titleFontSize=14,
).properties(
    width=900,
    height=300,
)